In [1]:
%reset
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
import imblearn
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
df_price_btc = pd.read_csv("price_BTC_EUR.csv",delimiter=";")

In [3]:
df_price_btc.head()

,Date,Time,Open,Close,Diference,Tendencia,bitcoin,blockchain,cryptocurrency,Ethereum,mining,Nvidia
0,22/04/2019,0:00,5303.36,5305.19,0.034,sin_tendencia,12,38,9,7,41,52
1,22/04/2019,1:00,5305.35,5303.19,-0.041,sin_tendencia,12,38,9,7,41,52
2,22/04/2019,2:00,5303.00,5280.26,-0.431,sin_tendencia,12,38,9,7,41,52
3,22/04/2019,3:00,5280.11,5276.84,-0.062,sin_tendencia,12,38,9,7,41,52
4,22/04/2019,4:00,5276.52,5268.05,-0.161,sin_tendencia,12,38,9,7,41,52


In [4]:
df_aux = df_price_btc
df_aux.head(5)

,Date,Time,Open,Close,Diference,Tendencia,bitcoin,blockchain,cryptocurrency,Ethereum,mining,Nvidia
0,22/04/2019,0:00,5303.36,5305.19,0.034,sin_tendencia,12,38,9,7,41,52
1,22/04/2019,1:00,5305.35,5303.19,-0.041,sin_tendencia,12,38,9,7,41,52
2,22/04/2019,2:00,5303.00,5280.26,-0.431,sin_tendencia,12,38,9,7,41,52
3,22/04/2019,3:00,5280.11,5276.84,-0.062,sin_tendencia,12,38,9,7,41,52
4,22/04/2019,4:00,5276.52,5268.05,-0.161,sin_tendencia,12,38,9,7,41,52


In [5]:
#dividimos x e y
#salidas = "Tendencia"
y = df_aux.Tendencia
x = df_aux.drop(columns=["Tendencia","Diference","Date","Time","Close"])

In [6]:
undersample = RandomOverSampler(sampling_strategy='minority')
x, y = undersample.fit_resample(x, y)

In [7]:
y = pd.factorize(y)[0]

In [8]:
min_max_scaler = preprocessing.MinMaxScaler()
datos_scaled = min_max_scaler.fit_transform(x)
x = pd.DataFrame(datos_scaled)

x = x.values
x

array([[0.01388898, 0.01754386, 0.41176471, ..., 0.02105263, 0.09756098,
        0.14285714],
       [0.01392204, 0.01754386, 0.41176471, ..., 0.02105263, 0.09756098,
        0.14285714],
       [0.013883  , 0.01754386, 0.41176471, ..., 0.02105263, 0.09756098,
        0.14285714],
       ...,
       [0.50495467, 0.94736842, 0.61764706, ..., 0.84210526, 0.6097561 ,
        0.42857143],
       [0.30441331, 0.35087719, 0.17647059, ..., 0.22105263, 0.2195122 ,
        0.51785714],
       [0.05832452, 0.12280702, 0.47058824, ..., 0.04210526, 0.09756098,
        0.05357143]])

In [9]:
aux_y = np.zeros((y.shape[0],3))
aux_y
for i in range(y.shape[0]):
    if y[i] == 0:
        aux_y[i]=[1,0,0]
    if y[i] == 1:
        aux_y[i]=[0,1,0]
    if y[i] == 2:
        aux_y[i]=[0,0,1]

In [10]:
x_train, x_test, y_train, y_test = train_test_split(x,aux_y,test_size=0.20)

In [11]:
#****** Crear modelo ******
model = Sequential()
# primera capa (entrada)
model.add(Dense(x.shape[1], activation = 'relu', input_dim = x.shape[1]))
# primera capa oculta
model.add(Dense(units = 3000, activation = 'sigmoid'))
# segunda capa oculta
model.add(Dense(units = 3000, activation = 'sigmoid'))
#capa de salida
model.add(Dense(3, activation = 'sigmoid'))
#
# definicion de la regla delta generalizada
opt = optimizers.SGD(learning_rate=0.2)
#
model.compile(loss='categorical_crossentropy', optimizer= opt, metrics = ['acc'])
#****** Entrenamiento ******
model.fit(x_train,y_train,validation_split = 0.0, epochs = 10,verbose=1)

Epoch 1/10
697/697 [==============================] - 21s 29ms/step - loss: 1.8431 - acc: 0.4517
Epoch 2/10
697/697 [==============================] - 21s 30ms/step - loss: 0.9387 - acc: 0.4527
Epoch 3/10
697/697 [==============================] - 21s 30ms/step - loss: 0.9358 - acc: 0.4597
Epoch 4/10
697/697 [==============================] - 21s 30ms/step - loss: 0.9393 - acc: 0.4499
Epoch 5/10
697/697 [==============================] - 21s 30ms/step - loss: 0.9349 - acc: 0.4636
Epoch 6/10
697/697 [==============================] - 21s 30ms/step - loss: 0.9291 - acc: 0.4728
Epoch 7/10
697/697 [==============================] - 21s 30ms/step - loss: 0.9260 - acc: 0.4951
Epoch 8/10
697/697 [==============================] - 21s 31ms/step - loss: 0.9241 - acc: 0.5134
Epoch 9/10
697/697 [==============================] - 21s 30ms/step - loss: 0.9099 - acc: 0.5368
Epoch 10/10
697/697 [==============================] - 21s 31ms/step - loss: 0.9107 - acc: 0.5387


In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 7)                 56        
_________________________________________________________________
dense_1 (Dense)              (None, 3000)              24000     
_________________________________________________________________
dense_2 (Dense)              (None, 3000)              9003000   
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 9003      
Total params: 9,036,059
Trainable params: 9,036,059
Non-trainable params: 0
_________________________________________________________________


In [13]:
y_pred = model.predict(x_test)
np.round(y_pred)
#confusion_matrix(y_test, y_pred)

array([[1., 0., 1.],
       [1., 0., 1.],
       [1., 0., 1.],
       ...,
       [1., 0., 1.],
       [1., 0., 1.],
       [1., 0., 1.]], dtype=float32)

In [14]:
y_pred

array([[0.72821146, 0.21129304, 0.5809344 ],
       [0.546854  , 0.2912137 , 0.6676855 ],
       [0.7406716 , 0.20565617, 0.5736641 ],
       ...,
       [0.7027532 , 0.2226657 , 0.5950684 ],
       [0.72185266, 0.21414897, 0.5845497 ],
       [0.67442185, 0.23515713, 0.60982114]], dtype=float32)

In [15]:
y_test

array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [16]:
model.evaluate(x_train,y_train)

697/697 [==============================] - 5s 7ms/step - loss: 0.9291 - acc: 0.5231


[0.9291027784347534, 0.5231003761291504]

In [17]:
model.evaluate(x_test,y_test)

175/175 [==============================] - 1s 7ms/step - loss: 0.9231 - acc: 0.5248


[0.9231104850769043, 0.5247578024864197]